In [11]:
import os
import pandas as  pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC


### File Retrieval and Preprocessing

In [64]:
files = os.listdir('../final_stats/preprocessed_3/')
files[:2]

def split_target(df, target_col='away_winner_wts'):
    df = df.copy()
    target = df.pop(target_col)
    return df, target

def normalize_df(df):
    scaler = StandardScaler()
    data, target = split_target(df)
    data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
    return data, target

def get_data(file):
    df = pd.read_csv(f'../final_stats/preprocessed/{file}', index_col=[0])
    data, target = normalize_df(df)
    return data, target


def remove_last_1(df):
    df = df.copy()
    df.drop(df.iloc[:, 1:52], inplace=True, axis=1)
    df.drop(df.iloc[:, 53:103], inplace=True, axis=1)
    return df


In [68]:
files

['preprocessed_no_sent_last_1.csv',
 'preprocessed_no_sent_last_3.csv',
 'preprocessed_no_sent_last_3_wt.csv',
 'preprocessed_no_sent_last_7.csv',
 'preprocessed_no_sent_last_7_wt.csv',
 'preprocessed_no_sent_last_ssn.csv',
 'preprocessed_no_sent_last_ssn_wt.csv',
 'preprocessed_sent_24_last_1.csv',
 'preprocessed_sent_24_last_3.csv',
 'preprocessed_sent_24_last_3_wt.csv',
 'preprocessed_sent_24_last_7.csv',
 'preprocessed_sent_24_last_7_wt.csv',
 'preprocessed_sent_24_last_ssn.csv',
 'preprocessed_sent_24_last_ssn_wt.csv',
 'preprocessed_sent_96_last_1.csv',
 'preprocessed_sent_96_last_3.csv',
 'preprocessed_sent_96_last_3_wt.csv',
 'preprocessed_sent_96_last_7.csv',
 'preprocessed_sent_96_last_7_wt.csv',
 'preprocessed_sent_96_last_ssn.csv',
 'preprocessed_sent_96_last_ssn_wt.csv',
 'preprocessed_sent_cross_last_1.csv',
 'preprocessed_sent_cross_last_3.csv',
 'preprocessed_sent_cross_last_3_wt.csv',
 'preprocessed_sent_cross_last_7.csv',
 'preprocessed_sent_cross_last_7_wt.csv',
 'pr

In [69]:
print(files[3])
x, y = get_data(files[3])
x.head()
y.head()

preprocessed_no_sent_last_7.csv


0    1
1    0
2    0
3    0
4    0
Name: away_winner_wts, dtype: int64

### Chart Code

In [6]:
def get_average_acc_history(acc_histories):
    num_epochs = len(acc_histories[0])
    return [np.mean([x[i] for x in acc_histories]) for i in range(num_epochs)]

def get_average_loss_history(loss_histories):
    num_epochs = len(loss_histories[0])
    return [np.mean([x[i] for x in loss_histories]) for i in range(num_epochs)]
        
def plot_ave_acc_and_loss_histories(acc_histories, loss_histories):
    average_acc_history = get_average_acc_history(acc_histories)
    average_loss_history = get_average_loss_history(loss_histories)

    plt.figure(figsize=(15,5))
    
    plt.subplot(1, 2, 1)
    plt.plot(range(1, len(average_acc_history) + 1), average_acc_history)
    plt.xlabel("Epochs")
    plt.ylabel("Validation Accuracy")

    plt.subplot(1, 2, 2)

    plt.plot(range(1, len(average_loss_history) + 1), average_loss_history)
    plt.xlabel("Epochs")
    plt.ylabel("Validation Loss")
    plt.show()

def compare_ave_loss_acc_histories(acc_hist_1, acc_hist_2, loss_hist_1, loss_hist_2):
    min_epochs = min(len(acc_hist_1[0]), len(acc_hist_2[0]))
    
    ave_acc_hist_1 = get_average_acc_history(acc_hist_1)
    ave_loss_hist_1 = get_average_loss_history(loss_hist_1)
    
    ave_acc_hist_2 = get_average_acc_history(acc_hist_2)
    ave_loss_hist_2 = get_average_loss_history(loss_hist_2)


    blue_dots = 'bo'
    solid_blue_line = 'b'
    red_dots = 'ro'
    solid_red_line = 'r'

    epochs = range(1, min_epochs + 1)
    plt.figure(figsize=(15,5))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, ave_acc_hist_1, solid_blue_line, label='Model 1')
    plt.plot(epochs, ave_acc_hist_2, solid_red_line, label='Model 2')
    plt.xlabel("Epochs")
    plt.ylabel("Validation Accuracy")
    plt.legend()

    plt.subplot(1, 2, 2)

    plt.plot(epochs, ave_loss_hist_1, solid_blue_line, label='Model 1')
    plt.plot(epochs, ave_loss_hist_2, solid_red_line, label='Model 2')
    plt.xlabel("Epochs")
    plt.ylabel("Validation Loss")
    plt.legend()
    
    plt.show()
    

In [6]:
from sklearn.model_selection import KFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

### Repeated K-Fold Cross Validation
Limited in dataset so want to run a few times
Grid search of the learning rate and the n-estimators perameter
No hold out test set but k=10 and n_repeats = 10 for a total of 100

In [67]:

n_ests = [10, 25, 50, 100, 300, 400, 500, 750, 1000]
learn_rates = [0.01, 0.1, 1, 2, 5, 10, 25, 50, 100, 200]
base_est_depths = [1]

def get_ada_classifier(n, lr, random_state=99):
    return AdaBoostClassifier(n_estimators=n, learning_rate=lr, random_state=random_state)

def grid_search(X_train, y_train, n_ests, learn_rates, base_depths, base_est='dt'):
    max_score = {
        'scores_mean': 0
    }
    
    all_scores = []
    
    for n in n_ests:
        for learn_rate in learn_rates:
            for depth in base_depths:
                if base_est == 'dt':
                    base = DecisionTreeClassifier(max_depth=depth)
                elif base_est == 'svm':
                    base = SVC(probability=True, kernel='linear')

                clf = AdaBoostClassifier(
                    base_estimator = base,
                    n_estimators = n, 
                    learning_rate = learn_rate, 
                    random_state = 99
                )

                cv = RepeatedKFold(n_splits=10, n_repeats=10, random_state=1)

                scores = cross_val_score(
                    clf, 
                    X_train, 
                    y_train, 
                    scoring='accuracy', 
                    cv=cv, 
                    n_jobs=-1
                )
                
                scores_dict = {
                    'scores_mean': scores.mean(),
                    'scores_std': scores.std(),
                    'n': n,
                    'depth': depth,
                    'learn_rate': learn_rate,
                    'scores': scores
                }
                all_scores.append(scores_dict)

                if scores.mean() > max_score['scores_mean']:
                    max_score = scores_dict
    return all_scores, max_score


Test n_ests holding learn rate and base depth stable. Find n_est 500 best. 
Hold n_ests at 500. Hold Base_est_depth. Test Learn rates.
Best learning rate = 10 was highest lr so check new lrs.
Best LR 15 again so increase learning rate spectrum.
Learning rate getting very large so just scrapped and doing full grid search to see how learning rate and n_ests interact. Sklearn notes a tradeoff between these two params


n_est initial test found best accuracy at max n_etsts so increase options and retest

In [72]:
x, y = get_data(files[3])
all_scores, max_score = grid_search(x,y, n_ests, learn_rates, [1], 'dt')
all_scores

[{'scores_mean': 0.444090909090909,
  'scores_std': 0.13863785394131917,
  'n': 10,
  'depth': 1,
  'learn_rate': 0.01,
  'scores': array([0.36363636, 0.45454545, 0.36363636, 0.63636364, 0.45454545,
         0.2       , 0.6       , 0.4       , 0.3       , 0.5       ,
         0.27272727, 0.36363636, 0.45454545, 0.36363636, 0.45454545,
         0.6       , 0.4       , 0.6       , 0.7       , 0.6       ,
         0.45454545, 0.36363636, 0.27272727, 0.45454545, 0.45454545,
         0.5       , 0.3       , 0.4       , 0.7       , 0.3       ,
         0.63636364, 0.54545455, 0.54545455, 0.27272727, 0.72727273,
         0.3       , 0.4       , 0.6       , 0.5       , 0.3       ,
         0.27272727, 0.54545455, 0.45454545, 0.45454545, 0.36363636,
         0.5       , 0.3       , 0.4       , 0.5       , 0.4       ,
         0.63636364, 0.45454545, 0.45454545, 0.36363636, 0.36363636,
         0.3       , 0.1       , 0.6       , 0.5       , 0.5       ,
         0.63636364, 0.54545455, 0.5454545

In [39]:
def get_best_params(all_scores):
    ms = max(all_scores, key=lambda x:x['scores_mean'])
    print(f"Accuracy: {round(ms['scores_mean']*100,2)}%")
    print(f"Best Paramaters: \nn_estimators: {ms['n']} \t learning_rate: {ms['learn_rate']} \t depth: {ms['depth']}")

In [73]:
get_best_params(all_scores)

Accuracy: 64.15%
Best Paramaters: 
n_estimators: 1000 	 learning_rate: 0.1 	 depth: 1


### Full Grid Search for DT Base
Ran on dataset of last 7 games no sentiment  

```n_ests = [10, 25, 50, 100, 300, 400, 500, 750, 1000]  ```  
```learn_rates = [0.01, 0.1, 1, 2, 5, 10, 25, 50, 100, 200]  ```  
```base_est_depths = 1```  

### Best Result DT Base:
#### Accuracy: **64.15%**  
Best Paramaters:   
n_estimators: 1000  
learning_rate: 0.1  
depth: 1

In [74]:
all_scores2, max_score2 = grid_search(x,y, [1000, 1250, 1500, 2000], [0.01, 0.1, 1], [1], 'dt')


In [77]:
all_scores3, max_score3 = grid_search(x,y, [2500, 3000], [0.05, 0.1, 0.5], [1], 'dt')

KeyboardInterrupt: 

In [75]:
get_best_params(all_scores2)

Accuracy: 64.51%
Best Paramaters: 
n_estimators: 2000 	 learning_rate: 0.1 	 depth: 1


In [60]:
get_best_params(all_scores_svm)

Accuracy: 40.78%
Best Paramaters: 
n_estimators: 600 	 learning_rate: 0.01 	 depth: 1


### Full Grid Search for SVM Base
```n_ests = [10, 25, 50, 100, 300, 400, 500, 600, 700]  ```  
```learn_rates = [0.01, 0.1, 1, 2, 5, 10, 25, 50, 100, 200]  ```  
```base_est_depths = 1```  

### Best Result SVM Base:
#### Accuracy: **40.78%**  
Best Paramaters:   
n_estimators: 600  
learning_rate: 0.01  
depth: 1

Decision Tree Base much better, use that for further models.

### Run for every dataset on base model??


In [61]:
def all_datasets(files):
    n_ests = [200, 500, 1000, ]
    learn_rates = [0.01, 0.1, 5, 30, 100, 500]
    base_est_depths = [1]

    results = []

    for f in files:
        x, y = get_data(f)
        all_scores, max_score = grid_search(x,y, n_ests, learn_rates, base_est_depths, 'dt')
        max_score['file'] = f
        results.append(max_score)
    return results

In [63]:
no_sent_files = files[:7]
no_sent_files

['preprocessed_no_sent_last_1.csv',
 'preprocessed_no_sent_last_10.csv',
 'preprocessed_no_sent_last_3.csv',
 'preprocessed_no_sent_last_5.csv',
 'preprocessed_no_sent_last_7.csv',
 'preprocessed_sent_24_last_1.csv',
 'preprocessed_sent_24_last_10.csv']

In [ ]:
no_sent_results = all_datasets(no_sent_files)

In [ ]:
df_no_sent_results = pd.DataFrame.from_records(no_sent_results, index ='file').sort_values('scores_mean', ascending=False)
df_no_sent_results

In [ ]:
column_dict_no_sent = {
    'preprocessed_no_sent_last_7.csv': '7 Games',
    'preprocessed_no_sent_last_7_wt.csv': '7 Games Wtd',
    'preprocessed_no_sent_last_1.csv': '1 Game',
    'preprocessed_no_sent_last_3.csv': '3 Games',
    'preprocessed_no_sent_last_3_wt.csv': '3 Games Wtd',
    'preprocessed_no_sent_last_ssn.csv': 'Season',
    'preprocessed_no_sent_last_ssn_wt.csv': 'Season Weighted',
}
def get_box_plot(df, col_labels):
    scores = df['scores'].apply(pd.Series).transpose()
    print(scores.columns)
    scores = scores.rename(columns=col_labels)
    scores.boxplot(rot=90, figsize=(12,8), grid=False, fontsize=15)

get_box_plot(df_no_sent_results, column_dict_no_sent)

In [41]:
def all_datasets(files):
    n_ests = [10, 50, 100, 200, 300]
    learn_rates = [0.01, 0.05, 0.1, 1, 5]
    base_est_depths = list(range(1,4,1))

    results = []

    for file in files:
        x, y = get_data(file)
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

        all_scores, max_score = grid_search(X_train, y_train, n_ests, learn_rates, base_est_depths)

        print('Data: ', file)
        test_score = train_base_model(max_score, X_train, y_train, X_test, y_test)

        results.append([file, test_score, max_score])

    return results

results = all_datasets(files)


Data:  preprocessed_no_sent_last_1.csv
Final Accuracy: 57.14%
Data:  preprocessed_no_sent_last_10.csv
Final Accuracy: 57.14%
Data:  preprocessed_no_sent_last_3.csv
Final Accuracy: 42.86%
Data:  preprocessed_no_sent_last_5.csv
Final Accuracy: 66.67%
Data:  preprocessed_no_sent_last_7.csv
Final Accuracy: 61.9%
Data:  preprocessed_sent_24_last_1.csv
Final Accuracy: 57.14%
Data:  preprocessed_sent_24_last_10.csv
Final Accuracy: 57.14%
Data:  preprocessed_sent_24_last_3.csv
Final Accuracy: 42.86%
Data:  preprocessed_sent_24_last_5.csv
Final Accuracy: 52.38%
Data:  preprocessed_sent_24_last_7.csv
Final Accuracy: 61.9%
Data:  preprocessed_sent_96_last_1.csv
Final Accuracy: 71.43%
Data:  preprocessed_sent_96_last_10.csv
Final Accuracy: 57.14%
Data:  preprocessed_sent_96_last_3.csv
Final Accuracy: 42.86%
Data:  preprocessed_sent_96_last_5.csv
Final Accuracy: 57.14%
Data:  preprocessed_sent_96_last_7.csv
Final Accuracy: 61.9%
Data:  preprocessed_sent_cross_last_1.csv
Final Accuracy: 57.14%
Data:

In [47]:
def get_top_score(results):
    top_score = 0
    top_index = 0
    for i, result in enumerate(results):
        # print(f'Test Accuracy: {result[1]}')
        if result[1] >= top_score:
            top_score = result[1]
            top_index = i
            print(i, top_score)

    print('Top Score: ', top_score, '\t Top Data: ', results[top_index][0])
    return results[top_index]

get_top_score(results)

0 0.5714285714285714
1 0.5714285714285714
3 0.6666666666666666
10 0.7142857142857143
30 0.7142857142857143
Top Score:  0.7142857142857143 	 Top Data:  preprocessed_sent_cross_last_5.csvno_last_1


['preprocessed_sent_cross_last_5.csvno_last_1',
 0.7142857142857143,
 {'scores_mean': 0.5601851851851852,
  'scores_std': 0.18416385022891585,
  'n': 200,
  'depth': 1,
  'learn_rate': 0.1,
  'scores': array([0.44444444, 0.44444444, 0.88888889, 0.66666667, 0.5       ,
         0.25      , 0.75      , 0.5       , 0.625     , 0.625     ,
         0.66666667, 0.55555556, 0.33333333, 0.55555556, 0.75      ,
         0.5       , 0.5       , 0.5       , 0.75      , 0.625     ,
         0.77777778, 0.55555556, 0.22222222, 0.44444444, 0.25      ,
         0.875     , 0.25      , 0.875     , 0.625     , 0.5       ])}]

In [48]:
results[10][0]

'preprocessed_sent_96_last_1.csv'

### Old Chart Code Functions

In [63]:
def plot_train_val_loss(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(loss) + 1)

    blue_dots = 'bo'
    solid_blue_line = 'b'

    plt.plot(epochs, loss, blue_dots, label = 'Training loss')
    plt.plot(epochs, val_loss, solid_blue_line, label = 'Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()
    
def plot_train_val_acc(history):
    plt.clf()

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
        
    epochs = range(1, len(acc) + 1)

    blue_dots = 'bo'
    solid_blue_line = 'b'
        
    plt.plot(epochs, acc, blue_dots, label = 'Training Accuracy')
    plt.plot(epochs, val_acc, solid_blue_line, label = 'Validation Accuracy')
    plt.title('Training and validation acc')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

def plot_train_val_loss_acc(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(1, len(acc) + 1)
    blue_dots = 'bo'
    solid_blue_line = 'b'

    plt.figure(figsize=(15, 5))

    plt.subplot(1, 2, 1)
    # plt.plot(epochs_range, acc, label='Training Accuracy')
    # plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    
 
    plt.plot(epochs_range, acc, blue_dots, label = 'Training Accuracy')
    plt.plot(epochs_range, val_acc, solid_blue_line, label = 'Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, blue_dots, label='Training Loss')
    plt.plot(epochs_range, val_loss, solid_blue_line, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()
  
def compare_loss_and_acc(hist1, hist2, title1, title2): 
    min_epochs = min(len(hist1.history['loss']), len(hist2.history['loss']))
    
    train_loss_1 = hist1['loss'][:min_epochs]
    val_loss_1 = hist1['val_loss'][:min_epochs]

    train_loss_2 = hist2['loss'][:min_epochs]
    val_loss_2 = hist2['val_loss'][:min_epochs]


    train_acc_1 = hist1['accuracy'][:min_epochs]
    val_acc_1 = hist1['val_accuracy'][:min_epochs]

    train_acc_2 = hist2['accuracy'][:min_epochs]
    val_acc_2 = hist2['val_accuracy'][:min_epochs]

    epochs = range(1, min_epochs + 1)

    blue_dots = 'bo'
    solid_blue_line = 'b'
    red_dots = 'ro'
    solid_red_line = 'r'

    plt.figure(figsize=(15,5))
    plt.subplot(1, 2, 1)

    plt.plot(epochs, train_loss_1, blue_dots, label = f'{title1} Training Loss')
    plt.plot(epochs, val_loss_1, solid_blue_line, label = f'{title1} Validation Loss')
    plt.plot(epochs, train_loss_2, red_dots, label = f'{title2} Training Loss')
    plt.plot(epochs, val_loss_2, solid_red_line, label = f'{title2} Validation Loss')
    # plt.title('Loss With Dropout Layers')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_acc_1, blue_dots, label = f'{title1} Training Accuracy')
    plt.plot(epochs, val_acc_1, solid_blue_line, label = f'{title1} Validation Accuracy')
    plt.plot(epochs, train_acc_2, red_dots, label = f'{title2} Training Accuracy')
    plt.plot(epochs, val_acc_2, solid_red_line, label = f'{title2} Validation Accuracy')
    # plt.title('Accuracy With Dropout Layers')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')

    plt.show()

def compare_acc(hist1, hist2, title1, title2): 
    min_epochs = min(len(hist1.history['accuracy']), len(hist2.history['accuracy']))
    acc = hist1.history['accuracy'][:min_epochs]
    val_acc = hist1.history['val_accuracy'][:min_epochs]
    acc_drop = hist2.history['accuracy'][:min_epochs]
    val_acc_drop = hist2.history['val_accuracy'][:min_epochs]
    
    epochs = range(1, min_epochs + 1)

    blue_dots = 'bo'
    solid_blue_line = 'b'
    red_dots = 'ro'
    solid_red_line = 'r'

    plt.plot(epochs, acc, blue_dots, label = f'{title1} Training Accuracy')
    plt.plot(epochs, val_acc, solid_blue_line, label = f'{title1} Validation Accuracy')
    plt.plot(epochs, acc_drop, red_dots, label = f'{title2} Training Accuracy')
    plt.plot(epochs, val_acc_drop, solid_red_line, label = f'{title2} Validation Accuracy')
    # plt.title('Accuracy With Dropout Layers')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()